In [1]:
from IPython import get_ipython
from IPython.core import magic_arguments
from IPython.core.magic import Magics, cell_magic, magics_class
from IPython.display import display
from IPython.utils.capture import capture_output
from pprint import pprint
from pathlib import Path

In [2]:
@magics_class
class CaptureMagic(Magics):
    @magic_arguments.magic_arguments()
    @magic_arguments.argument(
        "--path",
        "-p",
        default=None,
        help=(
            "The path where the video will be saved to. When there is more then one video, multiple paths have to be defined"
        ),
    )
    @cell_magic
    def capture_video(self, line, cell):
        args = magic_arguments.parse_argstring(CaptureMagic.capture_video, line)
        paths = args.path.strip('"').split(" ")
        with capture_output(stdout=False, stderr=False, display=True) as result:
            self.shell.run_cell(cell)
        for output in result.outputs:
            display(output) # only disabled for debugging
            global data # for debugging 
            data = output.data

            pprint(data) # for debugging 

            print("#####")
            if "text/html" in data: # this is not nice, is there any better way to access IPython.core.display.Video object ?
                path = paths.pop(0)
                if not path:
                    raise ValueError("Too few paths given!")
                video_object = data["text/html"]
                split_string = video_object.split('"')
                video_url = split_string[1]
                print(video_url) # for debugging 
                print(path) # for debugging 

                dest = Path(path)
                src = Path(video_url)
                dest.write_bytes(src.read_bytes())
                

                
ipy = get_ipython()
ipy.register_magics(CaptureMagic)

# Manim Example

In [3]:
from manim import *
param= "-v WARNING  --progress_bar None  -r  500,200  --disable_caching Example"
config.media_embed = False

Manim Community v0.17.2

In [4]:
%%capture_video -p "fromManim.mp4"
%%manim $param
class Example(Scene):
    def construct(self):
        dot= Dot(color= YELLOW, radius=0.5)
        self.add(dot)
        self.wait() # works!

{'text/html': '<video src="media/jupyter/Example@2023-03-12@17-12-33.mp4" '
              'controls autoplay loop style="max-width: 60%;"  >\n'
              '      Your browser does not support the <code>video</code> '
              'element.\n'
              '    </video>',
 'text/plain': '<IPython.core.display.Video object>'}
#####
media/jupyter/Example@2023-03-12@17-12-33.mp4
fromManim.mp4


In [5]:
config.media_embed = True

In [6]:
%%capture_video -p "fromManimEmbedded.mp4"
%%manim $param
class Example(Scene):
    def construct(self):
        dot= Dot(color= YELLOW, radius=0.5)
        self.add(dot)
        self.wait() # does not work!

{'text/html': '<video controls autoplay loop style="max-width: 60%;"  >\n'
              ' <source '
              'src="data:video/mp4;base64,AAAAIGZ0eXBpc29tAAACAGlzb21pc28yYXZjMW1wNDEAAAAIZnJlZQAACI9tZGF0AAACrgYF//+q3EXpvebZSLeWLNgg2SPu73gyNjQgLSBjb3JlIDE2NCByMzA5NSBiYWVlNDAwIC0gSC4yNjQvTVBFRy00IEFWQyBjb2RlYyAtIENvcHlsZWZ0IDIwMDMtMjAyMiAtIGh0dHA6Ly93d3cudmlkZW9sYW4ub3JnL3gyNjQuaHRtbCAtIG9wdGlvbnM6IGNhYmFjPTEgcmVmPTMgZGVibG9jaz0xOjA6MCBhbmFseXNlPTB4MzoweDExMyBtZT1oZXggc3VibWU9NyBwc3k9MSBwc3lfcmQ9MS4wMDowLjAwIG1peGVkX3JlZj0xIG1lX3JhbmdlPTE2IGNocm9tYV9tZT0xIHRyZWxsaXM9MSA4eDhkY3Q9MSBjcW09MCBkZWFkem9uZT0yMSwxMSBmYXN0X3Bza2lwPTEgY2hyb21hX3FwX29mZnNldD0tMiB0aHJlYWRzPTYgbG9va2FoZWFkX3RocmVhZHM9MSBzbGljZWRfdGhyZWFkcz0wIG5yPTAgZGVjaW1hdGU9MSBpbnRlcmxhY2VkPTAgYmx1cmF5X2NvbXBhdD0wIGNvbnN0cmFpbmVkX2ludHJhPTAgYmZyYW1lcz0zIGJfcHlyYW1pZD0yIGJfYWRhcHQ9MSBiX2JpYXM9MCBkaXJlY3Q9MSB3ZWlnaHRiPTEgb3Blbl9nb3A9MCB3ZWlnaHRwPTIga2V5aW50PTI1MCBrZXlpbnRfbWluPTI1IHNjZW5lY3V0PTQwIGludHJhX3JlZnJlc2g9MCByY19sb29rY

FileNotFoundError: [Errno 2] No such file or directory: 'max-width: 60%;'

# Web Example

In [7]:
%%capture_video -p "fromWeb.mp4"
from IPython.display import Video
Video('https://github.com/kolibril13/jupyter_video_presentation/blob/master/001.mp4?raw=true' ,width=300) # does not work

{'text/html': '<video '
              'src="https://github.com/kolibril13/jupyter_video_presentation/blob/master/001.mp4?raw=true" '
              'controls  width="300" >\n'
              '      Your browser does not support the <code>video</code> '
              'element.\n'
              '    </video>',
 'text/plain': '<IPython.core.display.Video object>'}
#####
https://github.com/kolibril13/jupyter_video_presentation/blob/master/001.mp4?raw=true
fromWeb.mp4


FileNotFoundError: [Errno 2] No such file or directory: 'https:/github.com/kolibril13/jupyter_video_presentation/blob/master/001.mp4?raw=true'

In [8]:
from pathlib import Path
Path("fromManim.mp4").unlink()

try:
    Path("fromManimEmbedded.mp4").unlink()
except:
    pass



try:
    Path("fromWeb.mp4").unlink()
except:
    pass

